<a href="https://colab.research.google.com/github/etomaro/RL/blob/main/bumberman_ice_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ボンバーマンのiceゲームを様々なAIモデルでクリアする

ルール

・keyをすべて取得しないとゴールできない

・一度通ったますは二度踏めない




一旦stage1のみ

ゲーム状態
4×5×4(4行5列 + (keyの有無、レイヤーの有無、一度通った有無、何もないマス))


ゲーム描画のマーク

1. 通ってない何もないマス -> 空白, 状態=0

2. プレイヤー -> ☆, 状態=1

3. 一度通ったマス -> ✖, 状態=2

4. key -> △, 状態=3

※ プレイヤーがいるマスは一度通ったマスにする

アクションリスト

右=0, 上=1, 左=2, 下=3

In [72]:
import numpy as np

class Game():
    ACTIONS = [0, 1, 2, 3]  # 右、上、左、下
    # 移動の際に使う値
    act_dict = {
        '0': np.array([0,1]),
        '1': np.array([-1,0]),
        '2': np.array([0,-1]),
        '3': np.array([1,0])
    }
    def __init__(self):
        self.reset()
    
    # ゲーム開始
    def start(self):
        print(f"GAME START!")

        # ゲーム終了するまで繰り返す
        while True:
            self.render()
            # 行動を選択する
            action = self.input_action(self.player_pos)
            # 行動する
            state, reward, is_done, info = self.step(action)
            if is_done:
                # ゲーム終了
                print(info)
                break
    
    # ゲームをリセットする
    def reset(self):
        """
        ゲーム状態初期
          key
            (0,0), (0,4), (2,2)
        　player
            (3,2)
        """
        state = np.zeros(shape=(4,5,1))
        # keyの設定(なにもない: 0, player: 1, 踏済: 2, key: 3)
        state[0][0], state[0][4], state[2][2] = [3],[3],[3]
        # playerの設定
        state[3][2] = [1]
        self.state = state

        # playerのポジション
        self.player_pos = [3,2]

        # 何回目の行動化
        self.count = 0

        # keyの取得数
        self.get_key = 0

    # ゲームを描画する
    def render(self):
        template =\
"""
           --- 
          | G |
 ---  ---  ---  ---  --- 
| 0-0 | 0-1 | 0-2 | 0-3 | 0-4 |
 ---  ---  ---  ---  --- 
| 1-0 | 1-1 | 1-2 | 1-3 | 1-4 |
 ---  ---  ---  ---  --- 
| 2-0 | 2-1 | 2-2 | 2-3 | 2-4 |
 ---  ---  ---  ---  --- 
| 3-0 | 3-1 | 3-2 | 3-3 | 3-4 |
 ---  ---  ---  ---  --- 
"""
        # key->△, player->☆, 踏済マス->✖, 何もないマス->空白
        for x, i in enumerate(self.state):
            for y, j in enumerate(i):
                rep_str = "  "
                rep_idx = f"{x}-{y}"
                if j[0] == 1:  # playerの場合
                    rep_str = "☆"
                elif j[0] == 2:  # 踏済の場合
                    rep_str = "✖"
                elif j[0] == 3:  # マスがkeyの場合
                    rep_str = "△"
                # 置換
                template = template.replace(rep_idx, rep_str)
        
        # 出力
        print(f"\n\nturn = {str(self.count)}\n{template}")
        
    # 状態をセットする
    def set_state(self):
        pass
    
    # 状態を取得する
    def get_state(self):
        pass
    
    # 行動を入力する
    def input_action(self, player_pos):
        action_txt = ""
        actionable_list = self.get_actionable_list(player_pos)
        if 0 in actionable_list:
            action_txt += "右に進む: ０を入力してください\n"
        if 1 in actionable_list:
            action_txt += "上に進む: 1を入力してください\n"
        if 2 in actionable_list:
            action_txt += "左に進む: 2を入力してください\n"
        if 3 in actionable_list:
            action_txt += "下に進む: 3を入力してください\n"
        
        is_actionable_input = False 
        while not is_actionable_input:

            action = input(\
f"""
行動を入力してください
{action_txt}
""")
            if int(action) in actionable_list:
                is_actionable_input = True
            else:
                print("可能な行動が選択されませんでした。再度行動を選択してください\n")

        return int(action)

    # 行動する
    def step(self, action):
        """
        Return
          state
          reward
          is_done
          info
        """
        info = ""
        reward = 0  # 一旦0にしとく

        # 移動量を取得
        move_coord = self.act_dict[str(action)].copy()
        # 新しいプレイヤー位置
        pos_new = move_coord + self.player_pos

        # ゲーム終了判定
        is_done, is_clear = self.is_done(pos_new, self.state)
        if is_done and is_clear:
            info = "GAME CLEAR"
        elif is_done and not is_clear:
            info = "GAME OVER"

        # --更新--
        # 現在のプレイヤーのマスの行動する前のマスの状態を取得する
        x, y = pos_new[0], pos_new[1]
        pre_x, pre_y = self.player_pos[0], self.player_pos[1]

        pre_state_xy = self.state[x][y]
        # keyの場合
        if pre_state_xy[0] == 3:
            self.get_key += 1  # increment
        self.state[x][y][0] = 1  # 今のマスをplayerの状態に更新
        self.state[pre_x][pre_y][0] = 2  # 前のマスを踏済の状態に更新

        self.player_pos = pos_new  # playerの現在位置
        self.count += 1  # 行動回数

        # print("count: ", self.count)
        # print("player_pos: ", self.player_pos)
        # print(f"pre_x:{pre_x} pre_y:{pre_y}")
        # print("pre_state_xy: ", pre_state_xy)
        # print(f"x:{x} y:{y}")
        # print("state_xy: ", self.state[x][y])

        return self.state, reward, is_done, info

    # 可能な行動リストを取得する
    def get_actionable_list(self, player_pos):
        action_list = self.ACTIONS.copy()
        # 壁に衝突するアクションは選択できない
        if player_pos[0] == 0:
            action_list.remove(1)
        if player_pos[0] == 3:
            action_list.remove(3)
        if player_pos[1] == 0:
            action_list.remove(2)
        if player_pos[1] == 4:
            action_list.remove(0)
        
        return action_list
    
    # アクション可能か判定する
    def is_actionable(self, player_pos, action):
        actionable_list = self.get_actionable_list(player_pos)
        if action in actionable_list:
            return True
        else:
            return False

    # 報酬
    def reward(self):
        pass

    # ゲーム終了か判定
    def is_done(self, player_pos, pre_state):
        """
        ゲーム終了する条件
        GAME OVER
            1. 二度マスを踏む
            2. keyをすべて取得してない状態でゴール
        GAME CLEAR
            1. keyをすべて取得した状態でゴール
        
        Return
            ゲーム終了かどうか, ゲームクリアかどうか
        """
        # 現在のプレイヤーのマスの行動する前のマスの状態を取得する
        x, y = player_pos[0], player_pos[1]
        pre_state_xy = pre_state[x][y]

        # playerの位置がgoal(0,2)の場合
        if x == 0 and y == 2:
            if self.get_key == 3:
                return True, True  # ゲームクリア
            else:
                return True, False  # GAME OVER
        
        # 二度マスを踏んだかどうか
        if pre_state_xy[0] == 2:
            return True, False  # GAME OVER
        
        return False, False  # ゲーム続行
        

game = Game()
game.start()

GAME START!


turn = 0

           --- 
          | G |
 ---  ---  ---  ---  --- 
| △ |    |    |    | △ |
 ---  ---  ---  ---  --- 
|    |    |    |    |    |
 ---  ---  ---  ---  --- 
|    |    | △ |    |    |
 ---  ---  ---  ---  --- 
|    |    | ☆ |    |    |
 ---  ---  ---  ---  --- 


行動を入力してください
右に進む: ０を入力してください
上に進む: 1を入力してください
左に進む: 2を入力してください

4
可能な行動が選択されませんでした。再度行動を選択してください


行動を入力してください
右に進む: ０を入力してください
上に進む: 1を入力してください
左に進む: 2を入力してください

2


turn = 1

           --- 
          | G |
 ---  ---  ---  ---  --- 
| △ |    |    |    | △ |
 ---  ---  ---  ---  --- 
|    |    |    |    |    |
 ---  ---  ---  ---  --- 
|    |    | △ |    |    |
 ---  ---  ---  ---  --- 
|    | ☆ | ✖ |    |    |
 ---  ---  ---  ---  --- 



KeyboardInterrupt: ignored

---下記テスト---

In [ ]:
a = input("入力して")
print(a)
print(type(a))

入力して1
1
<class 'str'>


In [66]:
a = [[1,2]]
a[0][0] = 2
print(a)

[[2, 2]]


In [52]:
a = 2
def test(b):
    b += 1
    print("b: ", b)
test(a)
print("a: ", a)

b:  3
a:  2


In [6]:
import numpy as np

a = np.zeros(shape=(2,3,1))
b = np.zeros(shape=(2,3,1))
print(a)
# print(b)

[[[0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]]]


In [ ]:
x = "1"
y = "2"
rep = f"{x}-{y}"
print(rep)

1-2


In [32]:
print('requesting...', end='', flush=True)
print('au')



requesting...au


In [34]:
# python コンソール　上書き出力


import sys, time
for i in range(100):
    sys.stdout.write("%d" % i)
    sys.stdout.flush()
    time.sleep(0.1)

012345678910111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970

KeyboardInterrupt: ignored

In [36]:
"""
\r 先頭にカーソルを戻す
end='' 文字出力の最後を改行ではなくする

-> print出力が毎度リセットされたみたいになる
"""
print('aiu', end='')
print('\rkakiku')

kakiku


In [40]:
import os
import time
print('aiu')
print('kakiku')
time.sleep(2)
print("\033[2J")

aiu
kakiku



In [41]:
print('aiu')
print('cccc', end='')
print('\rkakiku')

aiu
kakiku


In [8]:
# ターミナル出力の上書き
import time

for i in range(10):
    print('\rNo, %d' % i, end='')
    time.sleep(0.5)

No, 9

In [ ]:
import numpy as np
result_V_all = np.zeros(shape=(5,5,3))
result_V_all[1][0] = [1,1,1]
print(result_V_all)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[1. 1. 1.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
